In [7]:
import glob
import os
import random
import spacy
import string
from sklearn.model_selection import train_test_split 
import pickle as pkl
import nltk
from nltk.util import ngrams
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
%matplotlib inline 

# Read Train Data

In [16]:
file_list_pos = glob.glob(os.path.join(os.getcwd(), "aclImdb/train/pos/", "*.txt"))
file_list_neg = glob.glob(os.path.join(os.getcwd(), "aclImdb/train/neg/", "*.txt"))

train = []

for file_path in file_list_pos:
    with open(file_path) as f_input:
        train.append([f_input.read(),1])
train
for file_path in file_list_neg:
    with open(file_path) as f_input:
        train.append([f_input.read(),0])
        
# print (train[random.randint(0, len(train) - 1)])

# Read Test Data

In [17]:
file_list_pos = glob.glob(os.path.join(os.getcwd(), "aclImdb/test/pos/", "*.txt"))
file_list_neg = glob.glob(os.path.join(os.getcwd(), "aclImdb/test/neg/", "*.txt"))

test = []

for file_path in file_list_pos:
    with open(file_path) as f_input:
        test.append([f_input.read(),1])

for file_path in file_list_neg:
    with open(file_path) as f_input:
        test.append([f_input.read(),0])
        
# print (test[random.randint(0, len(test) - 1)])

# Split train data into train and val data

In [18]:
X_train = [i[0] for i in train]
y_train = [i[1] for i in train]

X_test = [i[0] for i in test]
y_test = [i[1] for i in test]

In [19]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 5000, random_state = 42)

In [20]:
print(len(X_train),len(X_val),len(X_test))
print(len(y_train),len(y_val),len(y_test))

20000 5000 25000
20000 5000 25000


# Tokenizing

In [8]:
# Load English tokenizer, tagger, parser, NER and word vectors
tokenizer = spacy.load('en_core_web_sm')
punctuations = string.punctuation

# lowercase and remove punctuation
def tokenize(sent):
    tokens = tokenizer(sent)
    return [token.text.lower() for token in tokens if (token.text not in punctuations)]


def tokenize_dataset(dataset):
    token_dataset = []
    # we are keeping track of all tokens in dataset 
    # in order to create vocabulary later
    all_tokens = []
    
    for sample in dataset:
        tokens = tokenize(sample)
        token_dataset.append(tokens)
        all_tokens += tokens

    return token_dataset, all_tokens

In [9]:
# # val set tokens
# print ("Tokenizing val data")
# val_data_tokens, _ = tokenize_dataset(X_val)
# pkl.dump(val_data_tokens, open("val_data_tokens.p", "wb"))

# # test set tokens
# print ("Tokenizing test data")
# test_data_tokens, _ = tokenize_dataset(X_test)
# pkl.dump(test_data_tokens, open("test_data_tokens.p", "wb"))

# # train set tokens
# print ("Tokenizing train data")
# train_data_tokens, all_train_tokens = tokenize_dataset(X_train)
# pkl.dump(train_data_tokens, open("train_data_tokens.p", "wb"))
# pkl.dump(all_train_tokens, open("all_train_tokens.p", "wb"))

In [8]:
# reload pickle
train_data_tokens = pkl.load(open("train_data_tokens.p", "rb")) 
### each sentence is lowercase and remove punctuation and there is a big list of smaller list
### each small list is the split words of the ogirinal text sentect
all_train_tokens = pkl.load(open("all_train_tokens.p", "rb"))  ### all unigram vocubular

val_data_tokens = pkl.load(open("val_data_tokens.p", "rb"))
test_data_tokens = pkl.load(open("test_data_tokens.p", "rb"))

# N-grams

In [9]:
max_vocab_size = 10000
# save index 1 for unk and 0 for pad
PAD_IDX = 0
UNK_IDX = 1

def build_vocab(all_tokens,n):
    ## This function is to transfer unigram to n gram and return mapping criteria from token to index 
    # all_tokens = unigram token vocabulary 
    # n is n_grams
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    all_tokens = [' '.join(i) for i in ngrams(all_train_tokens,n)]  ## ngrams comes from NLTK
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(max_vocab_size))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return token2id, id2token   # token2id is the most important one

In [12]:
# convert token to id in the dataset
def token2index_dataset(tokens_data,all_train_tokens,n):
    token2id, id2token = build_vocab(all_train_tokens,n)
    
    indices_data = []
    for tokens in tokens_data:
        tokens = [' '.join(i) for i in ngrams(tokens,n)]
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data,id2token

In [13]:
train_data_indices,id2token = token2index_dataset(train_data_tokens,all_train_tokens,4)
val_data_indices,_ = token2index_dataset(val_data_tokens,all_train_tokens,4)
test_data_indices, _ = token2index_dataset(test_data_tokens,all_train_tokens,4)

# double checking
print ("Train dataset size is {}".format(len(train_data_indices)))
print ("Val dataset size is {}".format(len(val_data_indices)))
print ("Test dataset size is {}".format(len(test_data_indices)))

Train dataset size is 20000
Val dataset size is 5000
Test dataset size is 25000


# Pytorch DataLoader

In [21]:
MAX_SENTENCE_LENGTH = 200

class ImdbDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """
    
    def __init__(self, data_list, target_list):
        """
        @param data_list: list of newsgroup tokens 
        @param target_list: list of newsgroup targets 

        """
        self.data_list = data_list
        self.target_list = target_list
        assert (len(self.data_list) == len(self.target_list))

    def __len__(self):
        return len(self.data_list)
        
    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        
        token_idx = self.data_list[key][:MAX_SENTENCE_LENGTH]
        label = self.target_list[key]
        return [token_idx, len(token_idx), label]

def imdb_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    data_list = []
    label_list = []
    length_list = []
    #print("collate batch: ", batch[0][0])
    #batch[0][0] = batch[0][0][:MAX_SENTENCE_LENGTH]
    for datum in batch:   ### batch = sample
        label_list.append(datum[2])
        length_list.append(datum[1])
    # padding
    for datum in batch:
        padded_vec = np.pad(np.array(datum[0]), 
                                pad_width=((0,MAX_SENTENCE_LENGTH-datum[1])),          ### 0代表左边没有pad,右边的值代表右边pad的个数
                                mode="constant", constant_values=0)
        data_list.append(padded_vec)
    return [torch.from_numpy(np.array(data_list)), torch.LongTensor(length_list), torch.LongTensor(label_list)]
#As another example, several loss functions like CrossEntropyLoss require that the target should be torch LongTensor. So before doing operations, make sure that your input Tensor types match the function definitions.


# create pytorch dataloader
#train_loader = NewsGroupDataset(train_data_indices, train_targets)
#val_loader = NewsGroupDataset(val_data_indices, val_targets)
#test_loader = NewsGroupDataset(test_data_indices, test_targets)

BATCH_SIZE = 32
train_dataset = ImdbDataset(train_data_indices, y_train)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=imdb_collate_func,   # collate_fn (callable, optional) – merges a list of samples to form a mini-batch.
                                           shuffle=True)
'''
- With data loader you can easily do :
- mini batches
- shuffling
- special sampling
- asynchronous loading with multiple workers
'''


val_dataset = ImdbDataset(val_data_indices, y_val)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=imdb_collate_func,
                                           shuffle=True)

test_dataset = ImdbDataset(test_data_indices, y_test)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=imdb_collate_func,
                                           shuffle=False)

#for i, (data, lengths, labels) in enumerate(train_loader):
#    print (data)
#    print (labels)
#    break

# Define bag-of-n-words model 

In [22]:
class BagOfWords(nn.Module):
    """
    BagOfWords classification model
    """
    def __init__(self, vocab_size, emb_dim):
        """
        @param vocab_size: size of the vocabulary. |V|
        @param emb_dim: size of the word embedding  d
        """
        super(BagOfWords, self).__init__()
        # pay attention to padding_idx 
        self.embed = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.linear = nn.Linear(emb_dim,20)   # nn.Linear(input, output)
    
    def forward(self, data, length):
        """
        
        @param data: matrix of size (batch_size, max_sentence_length). Each row in data represents a 
            review that is represented using n-gram index. Note that they are padded to have same length.
        @param length: an int tensor of size (batch_size), which represents the non-trivial (excludes padding)
            length of each sentences in the data.
        """
        out = self.embed(data)
        out = torch.sum(out, dim=1)
        out /= length.view(length.size()[0],1).expand_as(out).float()
     
        # return logits
        out = self.linear(out.float())
        return out


# Running Model

In [24]:
# Function for testing the model
def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    correct = 0
    total = 0
    model.eval()
    for data, lengths, labels in loader:
        data_batch, length_batch, label_batch = data, lengths, labels
        outputs = F.softmax(model(data_batch, length_batch), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]
        total += labels.size(0)  ### 一个batch有32个sample 所以labels.size(0) =32
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return (100 * correct / total)

# Hyper-parameter Tuning

In [25]:
'''
ngrams         predefined 1,2,3,4
max_vocab_size predefined 10000,20000
emb_dim        grid       100,200,300,400,500
optimizer      grid       'Adam','Sgd'
lr_rate        grid       0.0005,0.001,0.005
'''


def tuning(emb_dim, optimizer, learning_rate):
    BATCH_SIZE = 32
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                               batch_size=BATCH_SIZE,
                                               collate_fn=imdb_collate_func,   # collate_fn (callable, optional) – merges a list of samples to form a mini-batch.
                                               shuffle=True)

    val_dataset = ImdbDataset(val_data_indices, y_val)
    val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                               batch_size=BATCH_SIZE,
                                               collate_fn=imdb_collate_func,
                                               shuffle=True)

    test_dataset = ImdbDataset(test_data_indices, y_test)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                               batch_size=BATCH_SIZE,
                                               collate_fn=imdb_collate_func,
                                               shuffle=False)
    model = BagOfWords(len(id2token), emb_dim)
    num_epochs = 10 # number epoch to train

    # Criterion and Optimizer
    criterion = torch.nn.CrossEntropyLoss()  
    opt = 'torch.optim.'+ optimizer + '(model.parameters(), lr=learning_rate)'
    optimizer = eval(opt)

    for epoch in range(num_epochs):
        for i, (data, lengths, labels) in enumerate(train_loader):
            model.train()
            data_batch, length_batch, label_batch = data, lengths, labels
            optimizer.zero_grad()
            outputs = model(data_batch, length_batch)
            loss = criterion(outputs, label_batch)
    #         print(loss)
            loss.backward()
            optimizer.step()
            # validate every 100 iterations
            if i > 0 and i % 624 == 0:
                # validate
                val_acc = test_model(val_loader, model)
                print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format(
                    epoch+1, num_epochs, i+1, len(train_loader), val_acc))
    print(test_model(test_loader,model))

# ngram = 1, max_vocab_size = 10000

In [46]:
grid = {'emb_dim':[100,200,300,400,500],'optimizer':['Adam','RMSprop'], 'learning_rate': [0.0005,0.001,0.005]}
num_comb = len(ParameterGrid(grid))
count = 0

for j in range(len(ParameterGrid(grid))):
    count += 1
    print("Now it is {} out of {} combinations".format(count, num_comb))
    optimizer,learning_rate, emb_dim = ParameterGrid(grid)[j].values()
    myCsvRow = "optimizer = {}, learning_rate = {}, emb_dim = {}".format(optimizer, learning_rate, emb_dim)
    print(myCsvRow)
    tuning(emb_dim, optimizer, learning_rate)
    print()

Now it is 1 out of 30 combinations
optimizer = Adam, learning_rate = 0.0005, emb_dim = 100
Epoch: [1/10], Step: [625/625], Validation Acc: 64.5
Epoch: [2/10], Step: [625/625], Validation Acc: 73.34
Epoch: [3/10], Step: [625/625], Validation Acc: 78.58
Epoch: [4/10], Step: [625/625], Validation Acc: 80.74
Epoch: [5/10], Step: [625/625], Validation Acc: 82.54
Epoch: [6/10], Step: [625/625], Validation Acc: 83.62
Epoch: [7/10], Step: [625/625], Validation Acc: 84.42
Epoch: [8/10], Step: [625/625], Validation Acc: 84.76
Epoch: [9/10], Step: [625/625], Validation Acc: 85.16
Epoch: [10/10], Step: [625/625], Validation Acc: 85.68
85.072

Now it is 2 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.0005, emb_dim = 100
Epoch: [1/10], Step: [625/625], Validation Acc: 67.68
Epoch: [2/10], Step: [625/625], Validation Acc: 75.22
Epoch: [3/10], Step: [625/625], Validation Acc: 79.54
Epoch: [4/10], Step: [625/625], Validation Acc: 81.52
Epoch: [5/10], Step: [625/625], Validation Acc: 82.

Epoch: [9/10], Step: [625/625], Validation Acc: 86.62
Epoch: [10/10], Step: [625/625], Validation Acc: 86.7
85.552

Now it is 14 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.0005, emb_dim = 300
Epoch: [1/10], Step: [625/625], Validation Acc: 74.9
Epoch: [2/10], Step: [625/625], Validation Acc: 80.36
Epoch: [3/10], Step: [625/625], Validation Acc: 82.62
Epoch: [4/10], Step: [625/625], Validation Acc: 84.28
Epoch: [5/10], Step: [625/625], Validation Acc: 85.28
Epoch: [6/10], Step: [625/625], Validation Acc: 85.1
Epoch: [7/10], Step: [625/625], Validation Acc: 85.64
Epoch: [8/10], Step: [625/625], Validation Acc: 85.86
Epoch: [9/10], Step: [625/625], Validation Acc: 86.22
Epoch: [10/10], Step: [625/625], Validation Acc: 86.36
86.048

Now it is 15 out of 30 combinations
optimizer = Adam, learning_rate = 0.001, emb_dim = 300
Epoch: [1/10], Step: [625/625], Validation Acc: 79.78
Epoch: [2/10], Step: [625/625], Validation Acc: 84.18
Epoch: [3/10], Step: [625/625], Validation 

Epoch: [7/10], Step: [625/625], Validation Acc: 86.6
Epoch: [8/10], Step: [625/625], Validation Acc: 86.46
Epoch: [9/10], Step: [625/625], Validation Acc: 86.4
Epoch: [10/10], Step: [625/625], Validation Acc: 87.0
85.44

Now it is 27 out of 30 combinations
optimizer = Adam, learning_rate = 0.001, emb_dim = 500
Epoch: [1/10], Step: [625/625], Validation Acc: 81.8
Epoch: [2/10], Step: [625/625], Validation Acc: 85.02
Epoch: [3/10], Step: [625/625], Validation Acc: 86.36
Epoch: [4/10], Step: [625/625], Validation Acc: 86.9
Epoch: [5/10], Step: [625/625], Validation Acc: 86.78
Epoch: [6/10], Step: [625/625], Validation Acc: 86.74
Epoch: [7/10], Step: [625/625], Validation Acc: 86.32
Epoch: [8/10], Step: [625/625], Validation Acc: 86.3
Epoch: [9/10], Step: [625/625], Validation Acc: 85.84
Epoch: [10/10], Step: [625/625], Validation Acc: 85.36
83.2

Now it is 28 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.001, emb_dim = 500
Epoch: [1/10], Step: [625/625], Validation Acc: 82

# ngram = 2, max_vocab_size = 10000

In [51]:
grid = {'emb_dim':[100,200,300,400,500],'optimizer':['Adam','RMSprop'], 'learning_rate': [0.0005,0.001,0.005]}
num_comb = len(ParameterGrid(grid))
count = 0

for j in range(len(ParameterGrid(grid))):
    count += 1
    print("Now it is {} out of {} combinations".format(count, num_comb))
    optimizer,learning_rate, emb_dim = ParameterGrid(grid)[j].values()
    myCsvRow = "optimizer = {}, learning_rate = {}, emb_dim = {}".format(optimizer, learning_rate, emb_dim)
    print(myCsvRow)
    tuning(emb_dim, optimizer, learning_rate)
    print()

Now it is 1 out of 30 combinations
optimizer = Adam, learning_rate = 0.0005, emb_dim = 100
Epoch: [1/10], Step: [625/625], Validation Acc: 58.22
Epoch: [2/10], Step: [625/625], Validation Acc: 66.28
Epoch: [3/10], Step: [625/625], Validation Acc: 77.48
Epoch: [4/10], Step: [625/625], Validation Acc: 79.1
Epoch: [5/10], Step: [625/625], Validation Acc: 80.36
Epoch: [6/10], Step: [625/625], Validation Acc: 81.46
Epoch: [7/10], Step: [625/625], Validation Acc: 81.96
Epoch: [8/10], Step: [625/625], Validation Acc: 82.36
Epoch: [9/10], Step: [625/625], Validation Acc: 81.98
Epoch: [10/10], Step: [625/625], Validation Acc: 82.56
82.844

Now it is 2 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.0005, emb_dim = 100
Epoch: [1/10], Step: [625/625], Validation Acc: 65.48
Epoch: [2/10], Step: [625/625], Validation Acc: 71.54
Epoch: [3/10], Step: [625/625], Validation Acc: 75.66
Epoch: [4/10], Step: [625/625], Validation Acc: 77.74
Epoch: [5/10], Step: [625/625], Validation Acc: 78.

Epoch: [10/10], Step: [625/625], Validation Acc: 83.1
83.616

Now it is 14 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.0005, emb_dim = 300
Epoch: [1/10], Step: [625/625], Validation Acc: 61.6
Epoch: [2/10], Step: [625/625], Validation Acc: 76.28
Epoch: [3/10], Step: [625/625], Validation Acc: 79.4
Epoch: [4/10], Step: [625/625], Validation Acc: 80.54
Epoch: [5/10], Step: [625/625], Validation Acc: 81.88
Epoch: [6/10], Step: [625/625], Validation Acc: 81.16
Epoch: [7/10], Step: [625/625], Validation Acc: 82.94
Epoch: [8/10], Step: [625/625], Validation Acc: 83.22
Epoch: [9/10], Step: [625/625], Validation Acc: 83.2
Epoch: [10/10], Step: [625/625], Validation Acc: 83.2
83.628

Now it is 15 out of 30 combinations
optimizer = Adam, learning_rate = 0.001, emb_dim = 300
Epoch: [1/10], Step: [625/625], Validation Acc: 77.38
Epoch: [2/10], Step: [625/625], Validation Acc: 80.02
Epoch: [3/10], Step: [625/625], Validation Acc: 82.46
Epoch: [4/10], Step: [625/625], Validation Ac

Epoch: [8/10], Step: [625/625], Validation Acc: 82.98
Epoch: [9/10], Step: [625/625], Validation Acc: 83.24
Epoch: [10/10], Step: [625/625], Validation Acc: 83.1
83.656

Now it is 27 out of 30 combinations
optimizer = Adam, learning_rate = 0.001, emb_dim = 500
Epoch: [1/10], Step: [625/625], Validation Acc: 77.78
Epoch: [2/10], Step: [625/625], Validation Acc: 82.26
Epoch: [3/10], Step: [625/625], Validation Acc: 81.88
Epoch: [4/10], Step: [625/625], Validation Acc: 83.38
Epoch: [5/10], Step: [625/625], Validation Acc: 83.02
Epoch: [6/10], Step: [625/625], Validation Acc: 82.42
Epoch: [7/10], Step: [625/625], Validation Acc: 82.22
Epoch: [8/10], Step: [625/625], Validation Acc: 81.74
Epoch: [9/10], Step: [625/625], Validation Acc: 81.42
Epoch: [10/10], Step: [625/625], Validation Acc: 81.46
82.404

Now it is 28 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.001, emb_dim = 500
Epoch: [1/10], Step: [625/625], Validation Acc: 71.66
Epoch: [2/10], Step: [625/625], Validation

# ngram = 3, max_vocab_size = 10000

In [55]:
grid = {'emb_dim':[100,200,300,400,500],'optimizer':['Adam','RMSprop'], 'learning_rate': [0.0005,0.001,0.005]}
num_comb = len(ParameterGrid(grid))
count = 0

for j in range(len(ParameterGrid(grid))):
    count += 1
    print("Now it is {} out of {} combinations".format(count, num_comb))
    optimizer,learning_rate, emb_dim = ParameterGrid(grid)[j].values()
    myCsvRow = "optimizer = {}, learning_rate = {}, emb_dim = {}".format(optimizer, learning_rate, emb_dim)
    print(myCsvRow)
    tuning(emb_dim, optimizer, learning_rate)
    print()

Now it is 1 out of 30 combinations
optimizer = Adam, learning_rate = 0.0005, emb_dim = 100
Epoch: [1/10], Step: [625/625], Validation Acc: 54.5
Epoch: [2/10], Step: [625/625], Validation Acc: 56.42
Epoch: [3/10], Step: [625/625], Validation Acc: 59.48
Epoch: [4/10], Step: [625/625], Validation Acc: 66.72
Epoch: [5/10], Step: [625/625], Validation Acc: 74.6
Epoch: [6/10], Step: [625/625], Validation Acc: 74.1
Epoch: [7/10], Step: [625/625], Validation Acc: 71.0
Epoch: [8/10], Step: [625/625], Validation Acc: 75.66
Epoch: [9/10], Step: [625/625], Validation Acc: 75.88
Epoch: [10/10], Step: [625/625], Validation Acc: 76.08
76.692

Now it is 2 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.0005, emb_dim = 100
Epoch: [1/10], Step: [625/625], Validation Acc: 49.74
Epoch: [2/10], Step: [625/625], Validation Acc: 68.54
Epoch: [3/10], Step: [625/625], Validation Acc: 65.78
Epoch: [4/10], Step: [625/625], Validation Acc: 71.32
Epoch: [5/10], Step: [625/625], Validation Acc: 71.52


Epoch: [10/10], Step: [625/625], Validation Acc: 76.26
77.552

Now it is 14 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.0005, emb_dim = 300
Epoch: [1/10], Step: [625/625], Validation Acc: 57.06
Epoch: [2/10], Step: [625/625], Validation Acc: 53.1
Epoch: [3/10], Step: [625/625], Validation Acc: 58.36
Epoch: [4/10], Step: [625/625], Validation Acc: 74.08
Epoch: [5/10], Step: [625/625], Validation Acc: 70.26
Epoch: [6/10], Step: [625/625], Validation Acc: 76.14
Epoch: [7/10], Step: [625/625], Validation Acc: 76.54
Epoch: [8/10], Step: [625/625], Validation Acc: 73.84
Epoch: [9/10], Step: [625/625], Validation Acc: 74.22
Epoch: [10/10], Step: [625/625], Validation Acc: 75.8
76.704

Now it is 15 out of 30 combinations
optimizer = Adam, learning_rate = 0.001, emb_dim = 300
Epoch: [1/10], Step: [625/625], Validation Acc: 49.74
Epoch: [2/10], Step: [625/625], Validation Acc: 65.56
Epoch: [3/10], Step: [625/625], Validation Acc: 71.58
Epoch: [4/10], Step: [625/625], Validation

Epoch: [8/10], Step: [625/625], Validation Acc: 77.04
Epoch: [9/10], Step: [625/625], Validation Acc: 75.48
Epoch: [10/10], Step: [625/625], Validation Acc: 77.92
78.32

Now it is 27 out of 30 combinations
optimizer = Adam, learning_rate = 0.001, emb_dim = 500
Epoch: [1/10], Step: [625/625], Validation Acc: 51.02
Epoch: [2/10], Step: [625/625], Validation Acc: 74.9
Epoch: [3/10], Step: [625/625], Validation Acc: 75.92
Epoch: [4/10], Step: [625/625], Validation Acc: 76.16
Epoch: [5/10], Step: [625/625], Validation Acc: 75.54
Epoch: [6/10], Step: [625/625], Validation Acc: 77.78
Epoch: [7/10], Step: [625/625], Validation Acc: 76.74
Epoch: [8/10], Step: [625/625], Validation Acc: 77.56
Epoch: [9/10], Step: [625/625], Validation Acc: 77.18
Epoch: [10/10], Step: [625/625], Validation Acc: 76.2
76.944

Now it is 28 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.001, emb_dim = 500
Epoch: [1/10], Step: [625/625], Validation Acc: 52.48
Epoch: [2/10], Step: [625/625], Validation A

# ngram = 3, max_vocab_size = 20000

In [63]:
grid = {'emb_dim':[100,200,300,400,500],'optimizer':['Adam','RMSprop'], 'learning_rate': [0.0005,0.001,0.005]}
num_comb = len(ParameterGrid(grid))
count = 0

for j in range(len(ParameterGrid(grid))):
    count += 1
    print("Now it is {} out of {} combinations".format(count, num_comb))
    optimizer,learning_rate, emb_dim = ParameterGrid(grid)[j].values()
    myCsvRow = "optimizer = {}, learning_rate = {}, emb_dim = {}".format(optimizer, learning_rate, emb_dim)
    print(myCsvRow)
    tuning(emb_dim, optimizer, learning_rate)
    print()

Now it is 1 out of 30 combinations
optimizer = Adam, learning_rate = 0.0005, emb_dim = 100
Epoch: [1/10], Step: [625/625], Validation Acc: 56.4
Epoch: [2/10], Step: [625/625], Validation Acc: 50.2
Epoch: [3/10], Step: [625/625], Validation Acc: 61.06
Epoch: [4/10], Step: [625/625], Validation Acc: 74.26
Epoch: [5/10], Step: [625/625], Validation Acc: 74.16
Epoch: [6/10], Step: [625/625], Validation Acc: 73.3
Epoch: [7/10], Step: [625/625], Validation Acc: 76.82
Epoch: [8/10], Step: [625/625], Validation Acc: 76.98
Epoch: [9/10], Step: [625/625], Validation Acc: 76.5
Epoch: [10/10], Step: [625/625], Validation Acc: 78.58
78.8

Now it is 2 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.0005, emb_dim = 100
Epoch: [1/10], Step: [625/625], Validation Acc: 50.96
Epoch: [2/10], Step: [625/625], Validation Acc: 68.86
Epoch: [3/10], Step: [625/625], Validation Acc: 59.74
Epoch: [4/10], Step: [625/625], Validation Acc: 73.56
Epoch: [5/10], Step: [625/625], Validation Acc: 72.3
Epo

Epoch: [10/10], Step: [625/625], Validation Acc: 79.08
79.764

Now it is 14 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.0005, emb_dim = 300
Epoch: [1/10], Step: [625/625], Validation Acc: 53.98
Epoch: [2/10], Step: [625/625], Validation Acc: 71.8
Epoch: [3/10], Step: [625/625], Validation Acc: 55.5
Epoch: [4/10], Step: [625/625], Validation Acc: 73.18
Epoch: [5/10], Step: [625/625], Validation Acc: 74.06
Epoch: [6/10], Step: [625/625], Validation Acc: 76.86
Epoch: [7/10], Step: [625/625], Validation Acc: 77.56
Epoch: [8/10], Step: [625/625], Validation Acc: 76.68
Epoch: [9/10], Step: [625/625], Validation Acc: 78.06
Epoch: [10/10], Step: [625/625], Validation Acc: 79.08
79.464

Now it is 15 out of 30 combinations
optimizer = Adam, learning_rate = 0.001, emb_dim = 300
Epoch: [1/10], Step: [625/625], Validation Acc: 51.78
Epoch: [2/10], Step: [625/625], Validation Acc: 75.9
Epoch: [3/10], Step: [625/625], Validation Acc: 75.44
Epoch: [4/10], Step: [625/625], Validation 

Epoch: [8/10], Step: [625/625], Validation Acc: 78.58
Epoch: [9/10], Step: [625/625], Validation Acc: 78.62
Epoch: [10/10], Step: [625/625], Validation Acc: 76.96
77.328

Now it is 27 out of 30 combinations
optimizer = Adam, learning_rate = 0.001, emb_dim = 500
Epoch: [1/10], Step: [625/625], Validation Acc: 65.32
Epoch: [2/10], Step: [625/625], Validation Acc: 74.5
Epoch: [3/10], Step: [625/625], Validation Acc: 76.1
Epoch: [4/10], Step: [625/625], Validation Acc: 78.94
Epoch: [5/10], Step: [625/625], Validation Acc: 79.18
Epoch: [6/10], Step: [625/625], Validation Acc: 78.0
Epoch: [7/10], Step: [625/625], Validation Acc: 78.88
Epoch: [8/10], Step: [625/625], Validation Acc: 78.92
Epoch: [9/10], Step: [625/625], Validation Acc: 78.38
Epoch: [10/10], Step: [625/625], Validation Acc: 78.06
78.676

Now it is 28 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.001, emb_dim = 500
Epoch: [1/10], Step: [625/625], Validation Acc: 67.24
Epoch: [2/10], Step: [625/625], Validation A

# ngram = 2, max_vocab_size = 20000

In [67]:
grid = {'emb_dim':[100,200,300,400,500],'optimizer':['Adam','RMSprop'], 'learning_rate': [0.0005,0.001,0.005]}
num_comb = len(ParameterGrid(grid))
count = 0

for j in range(len(ParameterGrid(grid))):
    count += 1
    print("Now it is {} out of {} combinations".format(count, num_comb))
    optimizer,learning_rate, emb_dim = ParameterGrid(grid)[j].values()
    myCsvRow = "optimizer = {}, learning_rate = {}, emb_dim = {}".format(optimizer, learning_rate, emb_dim)
    print(myCsvRow)
    tuning(emb_dim, optimizer, learning_rate)
    print()

Now it is 1 out of 30 combinations
optimizer = Adam, learning_rate = 0.0005, emb_dim = 100
Epoch: [1/10], Step: [625/625], Validation Acc: 51.8
Epoch: [2/10], Step: [625/625], Validation Acc: 69.76
Epoch: [3/10], Step: [625/625], Validation Acc: 77.84
Epoch: [4/10], Step: [625/625], Validation Acc: 80.24
Epoch: [5/10], Step: [625/625], Validation Acc: 81.3
Epoch: [6/10], Step: [625/625], Validation Acc: 82.32
Epoch: [7/10], Step: [625/625], Validation Acc: 82.92
Epoch: [8/10], Step: [625/625], Validation Acc: 83.22
Epoch: [9/10], Step: [625/625], Validation Acc: 83.76
Epoch: [10/10], Step: [625/625], Validation Acc: 84.24
84.156

Now it is 2 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.0005, emb_dim = 100
Epoch: [1/10], Step: [625/625], Validation Acc: 54.92
Epoch: [2/10], Step: [625/625], Validation Acc: 69.04
Epoch: [3/10], Step: [625/625], Validation Acc: 77.92
Epoch: [4/10], Step: [625/625], Validation Acc: 79.08
Epoch: [5/10], Step: [625/625], Validation Acc: 80.3

Epoch: [10/10], Step: [625/625], Validation Acc: 84.62
84.656

Now it is 14 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.0005, emb_dim = 300
Epoch: [1/10], Step: [625/625], Validation Acc: 73.52
Epoch: [2/10], Step: [625/625], Validation Acc: 77.76
Epoch: [3/10], Step: [625/625], Validation Acc: 80.6
Epoch: [4/10], Step: [625/625], Validation Acc: 81.42
Epoch: [5/10], Step: [625/625], Validation Acc: 82.94
Epoch: [6/10], Step: [625/625], Validation Acc: 83.42
Epoch: [7/10], Step: [625/625], Validation Acc: 83.54
Epoch: [8/10], Step: [625/625], Validation Acc: 83.88
Epoch: [9/10], Step: [625/625], Validation Acc: 83.8
Epoch: [10/10], Step: [625/625], Validation Acc: 83.92
84.74

Now it is 15 out of 30 combinations
optimizer = Adam, learning_rate = 0.001, emb_dim = 300
Epoch: [1/10], Step: [625/625], Validation Acc: 73.14
Epoch: [2/10], Step: [625/625], Validation Acc: 82.76
Epoch: [3/10], Step: [625/625], Validation Acc: 83.54
Epoch: [4/10], Step: [625/625], Validation 

Epoch: [8/10], Step: [625/625], Validation Acc: 84.12
Epoch: [9/10], Step: [625/625], Validation Acc: 84.68
Epoch: [10/10], Step: [625/625], Validation Acc: 84.24
84.644

Now it is 27 out of 30 combinations
optimizer = Adam, learning_rate = 0.001, emb_dim = 500
Epoch: [1/10], Step: [625/625], Validation Acc: 79.8
Epoch: [2/10], Step: [625/625], Validation Acc: 82.98
Epoch: [3/10], Step: [625/625], Validation Acc: 83.88
Epoch: [4/10], Step: [625/625], Validation Acc: 84.18
Epoch: [5/10], Step: [625/625], Validation Acc: 84.04
Epoch: [6/10], Step: [625/625], Validation Acc: 83.82
Epoch: [7/10], Step: [625/625], Validation Acc: 83.06
Epoch: [8/10], Step: [625/625], Validation Acc: 82.8
Epoch: [9/10], Step: [625/625], Validation Acc: 82.94
Epoch: [10/10], Step: [625/625], Validation Acc: 82.68
83.42

Now it is 28 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.001, emb_dim = 500
Epoch: [1/10], Step: [625/625], Validation Acc: 80.32
Epoch: [2/10], Step: [625/625], Validation A

# ngram = 1, max_vocab_size = 20000

In [71]:
grid = {'emb_dim':[100,200,300,400,500],'optimizer':['Adam','RMSprop'], 'learning_rate': [0.0005,0.001,0.005]}
num_comb = len(ParameterGrid(grid))
count = 0

for j in range(len(ParameterGrid(grid))):
    count += 1
    print("Now it is {} out of {} combinations".format(count, num_comb))
    optimizer,learning_rate, emb_dim = ParameterGrid(grid)[j].values()
    myCsvRow = "optimizer = {}, learning_rate = {}, emb_dim = {}".format(optimizer, learning_rate, emb_dim)
    print(myCsvRow)
    tuning(emb_dim, optimizer, learning_rate)
    print()

Now it is 1 out of 30 combinations
optimizer = Adam, learning_rate = 0.0005, emb_dim = 100
Epoch: [1/10], Step: [625/625], Validation Acc: 64.98
Epoch: [2/10], Step: [625/625], Validation Acc: 74.06
Epoch: [3/10], Step: [625/625], Validation Acc: 79.68
Epoch: [4/10], Step: [625/625], Validation Acc: 82.0
Epoch: [5/10], Step: [625/625], Validation Acc: 83.1
Epoch: [6/10], Step: [625/625], Validation Acc: 84.34
Epoch: [7/10], Step: [625/625], Validation Acc: 84.92
Epoch: [8/10], Step: [625/625], Validation Acc: 85.44
Epoch: [9/10], Step: [625/625], Validation Acc: 85.76
Epoch: [10/10], Step: [625/625], Validation Acc: 86.16
85.588

Now it is 2 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.0005, emb_dim = 100
Epoch: [1/10], Step: [625/625], Validation Acc: 69.6
Epoch: [2/10], Step: [625/625], Validation Acc: 76.3
Epoch: [3/10], Step: [625/625], Validation Acc: 79.58
Epoch: [4/10], Step: [625/625], Validation Acc: 81.98
Epoch: [5/10], Step: [625/625], Validation Acc: 83.28


Epoch: [10/10], Step: [625/625], Validation Acc: 87.3
85.304

Now it is 14 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.0005, emb_dim = 300
Epoch: [1/10], Step: [625/625], Validation Acc: 75.7
Epoch: [2/10], Step: [625/625], Validation Acc: 81.06
Epoch: [3/10], Step: [625/625], Validation Acc: 83.3
Epoch: [4/10], Step: [625/625], Validation Acc: 84.94
Epoch: [5/10], Step: [625/625], Validation Acc: 85.18
Epoch: [6/10], Step: [625/625], Validation Acc: 86.14
Epoch: [7/10], Step: [625/625], Validation Acc: 86.92
Epoch: [8/10], Step: [625/625], Validation Acc: 87.36
Epoch: [9/10], Step: [625/625], Validation Acc: 87.26
Epoch: [10/10], Step: [625/625], Validation Acc: 87.3
85.648

Now it is 15 out of 30 combinations
optimizer = Adam, learning_rate = 0.001, emb_dim = 300
Epoch: [1/10], Step: [625/625], Validation Acc: 80.22
Epoch: [2/10], Step: [625/625], Validation Acc: 84.16
Epoch: [3/10], Step: [625/625], Validation Acc: 86.08
Epoch: [4/10], Step: [625/625], Validation A

Epoch: [8/10], Step: [625/625], Validation Acc: 86.96
Epoch: [9/10], Step: [625/625], Validation Acc: 87.24
Epoch: [10/10], Step: [625/625], Validation Acc: 87.3
85.28

Now it is 27 out of 30 combinations
optimizer = Adam, learning_rate = 0.001, emb_dim = 500
Epoch: [1/10], Step: [625/625], Validation Acc: 81.32
Epoch: [2/10], Step: [625/625], Validation Acc: 85.16
Epoch: [3/10], Step: [625/625], Validation Acc: 86.76
Epoch: [4/10], Step: [625/625], Validation Acc: 87.12
Epoch: [5/10], Step: [625/625], Validation Acc: 86.58
Epoch: [6/10], Step: [625/625], Validation Acc: 87.24
Epoch: [7/10], Step: [625/625], Validation Acc: 86.92
Epoch: [8/10], Step: [625/625], Validation Acc: 86.62
Epoch: [9/10], Step: [625/625], Validation Acc: 86.18
Epoch: [10/10], Step: [625/625], Validation Acc: 85.98
83.06

Now it is 28 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.001, emb_dim = 500
Epoch: [1/10], Step: [625/625], Validation Acc: 81.62
Epoch: [2/10], Step: [625/625], Validation A

# ngram = 4, max_vocab_size = 10000

In [26]:
grid = {'emb_dim':[100,200,300,400,500],'optimizer':['Adam','RMSprop'], 'learning_rate': [0.0005,0.001,0.005]}
num_comb = len(ParameterGrid(grid))
count = 0

for j in range(len(ParameterGrid(grid))):
    count += 1
    print("Now it is {} out of {} combinations".format(count, num_comb))
    optimizer,learning_rate, emb_dim = ParameterGrid(grid)[j].values()
    myCsvRow = "optimizer = {}, learning_rate = {}, emb_dim = {}".format(optimizer, learning_rate, emb_dim)
    print(myCsvRow)
    tuning(emb_dim, optimizer, learning_rate)
    print()

Now it is 1 out of 30 combinations
optimizer = Adam, learning_rate = 0.0005, emb_dim = 100
Epoch: [1/10], Step: [625/625], Validation Acc: 50.3
Epoch: [2/10], Step: [625/625], Validation Acc: 50.92
Epoch: [3/10], Step: [625/625], Validation Acc: 58.9
Epoch: [4/10], Step: [625/625], Validation Acc: 49.84
Epoch: [5/10], Step: [625/625], Validation Acc: 51.96
Epoch: [6/10], Step: [625/625], Validation Acc: 57.2
Epoch: [7/10], Step: [625/625], Validation Acc: 67.72
Epoch: [8/10], Step: [625/625], Validation Acc: 70.6
Epoch: [9/10], Step: [625/625], Validation Acc: 69.8
Epoch: [10/10], Step: [625/625], Validation Acc: 67.4
67.38

Now it is 2 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.0005, emb_dim = 100
Epoch: [1/10], Step: [625/625], Validation Acc: 49.7
Epoch: [2/10], Step: [625/625], Validation Acc: 55.7
Epoch: [3/10], Step: [625/625], Validation Acc: 52.92
Epoch: [4/10], Step: [625/625], Validation Acc: 56.68
Epoch: [5/10], Step: [625/625], Validation Acc: 55.94
Epoch

Epoch: [10/10], Step: [625/625], Validation Acc: 71.72
71.184

Now it is 14 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.0005, emb_dim = 300
Epoch: [1/10], Step: [625/625], Validation Acc: 50.78
Epoch: [2/10], Step: [625/625], Validation Acc: 49.76
Epoch: [3/10], Step: [625/625], Validation Acc: 60.56
Epoch: [4/10], Step: [625/625], Validation Acc: 50.8
Epoch: [5/10], Step: [625/625], Validation Acc: 70.24
Epoch: [6/10], Step: [625/625], Validation Acc: 62.66
Epoch: [7/10], Step: [625/625], Validation Acc: 65.4
Epoch: [8/10], Step: [625/625], Validation Acc: 69.74
Epoch: [9/10], Step: [625/625], Validation Acc: 70.64
Epoch: [10/10], Step: [625/625], Validation Acc: 68.32
68.66

Now it is 15 out of 30 combinations
optimizer = Adam, learning_rate = 0.001, emb_dim = 300
Epoch: [1/10], Step: [625/625], Validation Acc: 50.44
Epoch: [2/10], Step: [625/625], Validation Acc: 68.94
Epoch: [3/10], Step: [625/625], Validation Acc: 69.92
Epoch: [4/10], Step: [625/625], Validation 

Epoch: [8/10], Step: [625/625], Validation Acc: 70.86
Epoch: [9/10], Step: [625/625], Validation Acc: 64.78
Epoch: [10/10], Step: [625/625], Validation Acc: 67.4
67.56

Now it is 27 out of 30 combinations
optimizer = Adam, learning_rate = 0.001, emb_dim = 500
Epoch: [1/10], Step: [625/625], Validation Acc: 49.72
Epoch: [2/10], Step: [625/625], Validation Acc: 52.88
Epoch: [3/10], Step: [625/625], Validation Acc: 62.76
Epoch: [4/10], Step: [625/625], Validation Acc: 69.88
Epoch: [5/10], Step: [625/625], Validation Acc: 72.16
Epoch: [6/10], Step: [625/625], Validation Acc: 67.68
Epoch: [7/10], Step: [625/625], Validation Acc: 69.46
Epoch: [8/10], Step: [625/625], Validation Acc: 72.22
Epoch: [9/10], Step: [625/625], Validation Acc: 72.64
Epoch: [10/10], Step: [625/625], Validation Acc: 72.92
72.448

Now it is 28 out of 30 combinations
optimizer = RMSprop, learning_rate = 0.001, emb_dim = 500
Epoch: [1/10], Step: [625/625], Validation Acc: 60.28
Epoch: [2/10], Step: [625/625], Validation 